## Preprocesado de datos de población (INE)

Este notebook realiza el **primer análisis exploratorio y preprocesado de los datos de población** obtenidos del Instituto Nacional de Estadística (INE).

El objetivo es **limpiar, filtrar y transformar** los datos originales, que incluyen información por municipio, edad, sexo, nacionalidad y periodo, con el fin de obtener un conjunto de datos estructurado y adecuado para los análisis posteriores del proyecto.


- Instituto Nacional de Estadística (INE) – Población por municipios

In [1]:
import pandas as pd

### Carga de datos y exploración inicial

In [2]:
# Crear dataframe con datos ine
ine_raw = pd.read_csv('../data/raw/INE-poblacion-grupo-edad.csv', sep=';')

/tmp/ipykernel_47081/2371885041.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  ine_raw = pd.read_csv('../data/raw/INE-poblacion-grupo-edad.csv', sep=';')


> Al crear el *DataFrame* con los datos de población, **pandas muestra un `DtypeWarning`**, indicando que la **columna 7 (`Total`) contiene tipos mezclados** (valores numéricos y texto).
>
> Este aviso se debe a que la columna `Total` incluye valores numéricos con **separador de miles (`.`)**, por ejemplo `'49.128.297'`, que pandas no reconoce automáticamente como números y, por tanto, interpreta la columna como texto (`object`).

In [3]:
# Verificar valores no numéricos en la columna 'Total'
mask = pd.to_numeric(ine_raw['Total'], errors='coerce').isna()
ine_raw.loc[mask, 'Total'].value_counts()

Total
49.128.297    1
48.619.695    1
48.085.361    1
47.486.727    1
47.400.798    1
             ..
1.052.992     1
1.052.174     1
1.054.185     1
1.059.612     1
1.065.406     1
Name: count, Length: 715, dtype: int64

In [4]:
# Volver a cargar los datos especificando los parámetros correctos para miles y decimales
ine_raw = pd.read_csv(
    '../data/raw/INE-poblacion-grupo-edad.csv',
    sep=';',
    thousands='.',
    decimal=',',
    low_memory=False
)

#### Exploración del dataset

In [5]:
# Dimensión del dataset
ine_raw.shape

(8103150, 8)

In [6]:
# Tipos de datos
ine_raw.dtypes

Total Nacional     object
Provincias         object
Municipios         object
Sexo               object
Edad               object
Nacionalidad       object
Periodo             int64
Total             float64
dtype: object

In [7]:
# Valores nulos por columna
ine_raw.isna().sum()

Total Nacional        0
Provincias          990
Municipios        52470
Sexo                  0
Edad                  0
Nacionalidad          0
Periodo               0
Total               594
dtype: int64

In [8]:
# Valores únicos por columna
ine_raw.nunique()

Total Nacional        1
Provincias           52
Municipios         8132
Sexo                  3
Edad                 22
Nacionalidad          3
Periodo               5
Total             40974
dtype: int64

In [9]:
# Descripción estadística del dataset
ine_raw.describe(include='all')

,Total Nacional,Provincias,Municipios,Sexo,Edad,Nacionalidad,Periodo,Total
count,8103150,8102160,8050680,8103150,8103150,8103150,8.103150e+06,8.102556e+06
unique,1,52,8132,3,22,3,NaN,NaN
top,Total Nacional,09 Burgos,01001 Alegría-Dulantzi,Total,Todas las edades,Total,NaN,NaN
freq,8103150,368280,990,2701050,368325,2701050,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,2.023000e+03,7.130220e+02
std,NaN,NaN,NaN,NaN,NaN,NaN,1.414214e+00,6.547710e+04
min,NaN,NaN,NaN,NaN,NaN,NaN,2.021000e+03,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,2.022000e+03,1.000000e+00
50%,NaN,NaN,NaN,NaN,NaN,NaN,2.023000e+03,6.000000e+00
75%,NaN,NaN,NaN,NaN,NaN,NaN,2.024000e+03,4.100000e+01


In [10]:
# Vista previa del dataframe
ine_raw.sample(5)

,Total Nacional,Provincias,Municipios,Sexo,Edad,Nacionalidad,Periodo,Total
3863685,Total Nacional,25 Lleida,25063 Canejan,Mujeres,De 10 a 14 años,Total,2025,1.0
7424933,Total Nacional,47 Valladolid,47208 Villagómez la Nueva,Mujeres,De 80 a 84 años,Española,2022,1.0
4151543,Total Nacional,"26 Rioja, La","26122 Rasillo de Cameros, El",Hombres,De 40 a 44 años,Española,2022,6.0
7068287,Total Nacional,46 Valencia/València,46108 Chera,Mujeres,De 0 a 4 años,Total,2023,4.0
5575884,Total Nacional,37 Salamanca,37267 Retortillo,Total,De 60 a 64 años,Española,2021,17.0


### Análisis exploratorio y preparación de variables

In [11]:
# Renombrar columnas
ine_clean = ine_raw.rename(columns={
    'Total Nacional': 'total_nacional',
    'Provincias': 'provincia',
    'Municipios': 'municipio',
    'Sexo': 'sexo',
    'Edad': 'grupo_edad',
    'Nacionalidad': 'nacionalidad',
    'Periodo': 'anyo',
    'Total': 'poblacion'
})
ine_clean.describe(include='all')

,total_nacional,provincia,municipio,sexo,grupo_edad,nacionalidad,anyo,poblacion
count,8103150,8102160,8050680,8103150,8103150,8103150,8.103150e+06,8.102556e+06
unique,1,52,8132,3,22,3,NaN,NaN
top,Total Nacional,09 Burgos,01001 Alegría-Dulantzi,Total,Todas las edades,Total,NaN,NaN
freq,8103150,368280,990,2701050,368325,2701050,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,2.023000e+03,7.130220e+02
std,NaN,NaN,NaN,NaN,NaN,NaN,1.414214e+00,6.547710e+04
min,NaN,NaN,NaN,NaN,NaN,NaN,2.021000e+03,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,2.022000e+03,1.000000e+00
50%,NaN,NaN,NaN,NaN,NaN,NaN,2.023000e+03,6.000000e+00
75%,NaN,NaN,NaN,NaN,NaN,NaN,2.024000e+03,4.100000e+01


#### Variable `total_nacional`

La variable `total_nacional` presenta un único valor constante para todas las observaciones ("Total Nacional"), por lo que no aporta información adicional ni capacidad discriminatoria para el análisis.

Dado que el estudio se centra en el análisis de población a nivel municipal y su posterior integración con otras fuentes territoriales y sanitarias, esta variable se elimina del conjunto de datos.

In [12]:
# Quitar la columna 'total_nacional' ya que no aporta información relevante
ine_clean = ine_clean.drop(columns=['total_nacional'])
ine_clean.describe(include='all')

,provincia,municipio,sexo,grupo_edad,nacionalidad,anyo,poblacion
count,8102160,8050680,8103150,8103150,8103150,8.103150e+06,8.102556e+06
unique,52,8132,3,22,3,NaN,NaN
top,09 Burgos,01001 Alegría-Dulantzi,Total,Todas las edades,Total,NaN,NaN
freq,368280,990,2701050,368325,2701050,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,2.023000e+03,7.130220e+02
std,NaN,NaN,NaN,NaN,NaN,1.414214e+00,6.547710e+04
min,NaN,NaN,NaN,NaN,NaN,2.021000e+03,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,2.022000e+03,1.000000e+00
50%,NaN,NaN,NaN,NaN,NaN,2.023000e+03,6.000000e+00
75%,NaN,NaN,NaN,NaN,NaN,2.024000e+03,4.100000e+01


#### Variable `provincia`

La variable `provincia` contiene información compuesta por el código numérico de la provincia y su nombre, ambos en una única cadena de texto. Para facilitar el filtrado territorial y la posterior integración con otras fuentes de datos, se procede a separar esta variable en dos nuevas columnas:

- `cod_provincia`: código numérico de la provincia.
- `nombre_provincia`: nombre de la provincia.

Previamente, se eliminan las filas con valores nulos en esta variable, ya que no pueden ser utilizadas en un análisis territorial ni en procesos de integración de datos.

Una vez separadas las variables y eliminada la columna original, el conjunto de datos se filtra para conservar únicamente las provincias pertenecientes a la Comunitat Valenciana (Alicante/Alacant, Castellón/Castelló y Valencia/València), en coherencia con los objetivos del proyecto.


In [13]:
# Eliminar filas con valores nulos en la columna 'provincia'
ine_clean = ine_clean.dropna(subset=['provincia']).reset_index(drop=True)
ine_clean.describe(include='all')

,provincia,municipio,sexo,grupo_edad,nacionalidad,anyo,poblacion
count,8102160,8050680,8102160,8102160,8102160,8.102160e+06,8.101566e+06
unique,52,8132,3,22,3,NaN,NaN
top,09 Burgos,01001 Alegría-Dulantzi,Total,Todas las edades,Total,NaN,NaN
freq,368280,990,2700720,368280,2700720,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,2.023000e+03,4.754061e+02
std,NaN,NaN,NaN,NaN,NaN,1.414214e+00,1.555400e+04
min,NaN,NaN,NaN,NaN,NaN,2.021000e+03,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,2.022000e+03,1.000000e+00
50%,NaN,NaN,NaN,NaN,NaN,2.023000e+03,6.000000e+00
75%,NaN,NaN,NaN,NaN,NaN,2.024000e+03,4.100000e+01


In [14]:
# Separar la variable provincia en cod_provincia y nombre_provincia
ine_clean[['cod_provincia', 'nombre_provincia']] = ine_clean['provincia'].str.split(' ', n=1, expand=True)
ine_clean['cod_provincia'] = ine_clean['cod_provincia'].astype(int)
ine_clean.describe(include='all')

,provincia,municipio,sexo,grupo_edad,nacionalidad,anyo,poblacion,cod_provincia,nombre_provincia
count,8102160,8050680,8102160,8102160,8102160,8.102160e+06,8.101566e+06,8.102160e+06,8102160
unique,52,8132,3,22,3,NaN,NaN,NaN,52
top,09 Burgos,01001 Alegría-Dulantzi,Total,Todas las edades,Total,NaN,NaN,NaN,Burgos
freq,368280,990,2700720,368280,2700720,NaN,NaN,NaN,368280
mean,NaN,NaN,NaN,NaN,NaN,2.023000e+03,4.754061e+02,2.665872e+01,NaN
std,NaN,NaN,NaN,NaN,NaN,1.414214e+00,1.555400e+04,1.489402e+01,NaN
min,NaN,NaN,NaN,NaN,NaN,2.021000e+03,0.000000e+00,1.000000e+00,NaN
25%,NaN,NaN,NaN,NaN,NaN,2.022000e+03,1.000000e+00,1.300000e+01,NaN
50%,NaN,NaN,NaN,NaN,NaN,2.023000e+03,6.000000e+00,2.600000e+01,NaN
75%,NaN,NaN,NaN,NaN,NaN,2.024000e+03,4.100000e+01,4.100000e+01,NaN


In [15]:
# Quitar la columna provincia original
ine_clean = ine_clean.drop(columns=['provincia'])

In [16]:
ine_clean['cod_provincia'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52])

In [17]:
ine_clean['nombre_provincia'].unique()

array(['Araba/Álava', 'Albacete', 'Alicante/Alacant', 'Almería', 'Ávila',
       'Badajoz', 'Balears, Illes', 'Barcelona', 'Burgos', 'Cáceres',
       'Cádiz', 'Castellón/Castelló', 'Ciudad Real', 'Córdoba',
       'Coruña, A', 'Cuenca', 'Girona', 'Granada', 'Guadalajara',
       'Gipuzkoa', 'Huelva', 'Huesca', 'Jaén', 'León', 'Lleida',
       'Rioja, La', 'Lugo', 'Madrid', 'Málaga', 'Murcia', 'Navarra',
       'Ourense', 'Asturias', 'Palencia', 'Palmas, Las', 'Pontevedra',
       'Salamanca', 'Santa Cruz de Tenerife', 'Cantabria', 'Segovia',
       'Sevilla', 'Soria', 'Tarragona', 'Teruel', 'Toledo',
       'Valencia/València', 'Valladolid', 'Bizkaia', 'Zamora', 'Zaragoza',
       'Ceuta', 'Melilla'], dtype=object)

Debido a que nuestro estudio se centra en la Comunidad Valenciana, vamos a filtrar el dataset para quedarnos solo con las provincias de CV.

In [18]:
ine_clean = ine_clean[ine_clean['nombre_provincia'].isin(['Alicante/Alacant', 'Castellón/Castelló', 'Valencia/València'])].reset_index(drop=True)
ine_clean.describe(include='all')

,municipio,sexo,grupo_edad,nacionalidad,anyo,poblacion,cod_provincia,nombre_provincia
count,536580,539550,539550,539550,539550.000000,5.395500e+05,539550.000000,539550
unique,542,3,22,3,NaN,NaN,NaN,3
top,"03001 Atzúbia, l'",Total,Todas las edades,Total,NaN,NaN,NaN,Valencia/València
freq,990,179850,24525,179850,NaN,NaN,NaN,264330
mean,NaN,NaN,NaN,NaN,2023.000000,7.750663e+02,26.311927,NaN
std,NaN,NaN,NaN,NaN,1.414215,1.767557e+04,19.560368,NaN
min,NaN,NaN,NaN,NaN,2021.000000,0.000000e+00,3.000000,NaN
25%,NaN,NaN,NaN,NaN,2022.000000,3.000000e+00,3.000000,NaN
50%,NaN,NaN,NaN,NaN,2023.000000,2.000000e+01,12.000000,NaN
75%,NaN,NaN,NaN,NaN,2024.000000,1.160000e+02,46.000000,NaN


#### Variable `municipio`

La variable `municipio` también contiene información compuesta por el código numérico del municipio y su nombre, ambos en una única cadena de texto. Para facilitar el filtrado territorial y la posterior integración con otras fuentes de datos, se procede a separar esta variable en dos nuevas columnas:

- `cod_municipio`: código numérico del municipio.
- `nombre_municipio`: nombre del municipio.

Previamente, se eliminan las filas con valores nulos en esta variable, ya que no pueden ser utilizadas en un análisis territorial ni en procesos de integración de datos.

In [19]:
# Eliminar filas con valores nulos en la columna 'municipio'
ine_clean = ine_clean.dropna(subset=['municipio']).reset_index(drop=True)
ine_clean.describe(include='all')

,municipio,sexo,grupo_edad,nacionalidad,anyo,poblacion,cod_provincia,nombre_provincia
count,536580,536580,536580,536580,536580.000000,536580.000000,536580.000000,536580
unique,542,3,22,3,NaN,NaN,NaN,3
top,"03001 Atzúbia, l'",Total,Todas las edades,Total,NaN,NaN,NaN,Valencia/València
freq,990,178860,24390,178860,NaN,NaN,NaN,263340
mean,NaN,NaN,NaN,NaN,2023.000000,389.678169,26.345018,NaN
std,NaN,NaN,NaN,NaN,1.414215,4997.303637,19.560903,NaN
min,NaN,NaN,NaN,NaN,2021.000000,0.000000,3.000000,NaN
25%,NaN,NaN,NaN,NaN,2022.000000,3.000000,3.000000,NaN
50%,NaN,NaN,NaN,NaN,2023.000000,20.000000,12.000000,NaN
75%,NaN,NaN,NaN,NaN,2024.000000,112.000000,46.000000,NaN


In [20]:
# Separar la variable municipio en cod_municipio y nombre_municipio
ine_clean[['cod_municipio', 'nombre_municipio']] = (
    ine_clean['municipio']
    .str.split(' ', n=1, expand=True)
)
ine_clean['cod_municipio'] = ine_clean['cod_municipio'].astype(int)
ine_clean.describe(include='all')

,municipio,sexo,grupo_edad,nacionalidad,anyo,poblacion,cod_provincia,nombre_provincia,cod_municipio,nombre_municipio
count,536580,536580,536580,536580,536580.000000,536580.000000,536580.000000,536580,536580.000000,536580
unique,542,3,22,3,NaN,NaN,NaN,3,NaN,542
top,"03001 Atzúbia, l'",Total,Todas las edades,Total,NaN,NaN,NaN,Valencia/València,NaN,"Atzúbia, l'"
freq,990,178860,24390,178860,NaN,NaN,NaN,263340,NaN,990
mean,NaN,NaN,NaN,NaN,2023.000000,389.678169,26.345018,NaN,26459.627306,NaN
std,NaN,NaN,NaN,NaN,1.414215,4997.303637,19.560903,NaN,19586.070731,NaN
min,NaN,NaN,NaN,NaN,2021.000000,0.000000,3.000000,NaN,3001.000000,NaN
25%,NaN,NaN,NaN,NaN,2022.000000,3.000000,3.000000,NaN,3139.000000,NaN
50%,NaN,NaN,NaN,NaN,2023.000000,20.000000,12.000000,NaN,12139.500000,NaN
75%,NaN,NaN,NaN,NaN,2024.000000,112.000000,46.000000,NaN,46131.000000,NaN


In [21]:
# Quitar columna municipio original
ine_clean = ine_clean.drop(columns=['municipio'])

In [22]:
ine_clean[['cod_municipio', 'nombre_municipio']].sample(5)

,cod_municipio,nombre_municipio
313915,46042,Aielo de Malferit
278624,46006,Albaida
230768,12102,Santa Magdalena de Pulpis
494788,46224,Segart
499586,46229,Siete Aguas


El conjunto de datos contiene 542 municipios distintos de la Comunidad Valenciana.

#### Variable `sexo`

La variable `sexo` presenta tres categorías: `Total`, `Hombres` y `Mujeres`.  
Sin embargo, el objetivo principal de este estudio es analizar la **relación entre la estructura por grupos de edad de la población y la distribución territorial de los centros sanitarios**, a nivel municipal.

Mantener la desagregación por sexo implicaría:
- Incrementar innecesariamente la dimensionalidad del dataset
- Dificultar la integración con otros datasets territoriales
- Generar múltiples registros por municipio, alejándose del nivel de análisis definido

Por este motivo, en la fase de preprocesado se decide **conservar únicamente los registros correspondientes a `sexo = Total`** y eliminar la variable del dataset final, obteniendo así un único valor agregado de población por grupo de edad y municipio.

In [23]:
ine_clean = ine_clean[ine_clean['sexo'] == 'Total']
ine_clean.drop(columns='sexo', inplace=True)
ine_clean.describe(include='all')

,grupo_edad,nacionalidad,anyo,poblacion,cod_provincia,nombre_provincia,cod_municipio,nombre_municipio
count,178860,178860,178860.000000,178860.000000,178860.000000,178860,178860.000000,178860
unique,22,3,NaN,NaN,NaN,3,NaN,542
top,Todas las edades,Total,NaN,NaN,NaN,Valencia/València,NaN,"Atzúbia, l'"
freq,8130,59620,NaN,NaN,NaN,87780,NaN,330
mean,NaN,NaN,2023.000000,584.517254,26.345018,NaN,26459.627306,NaN
std,NaN,NaN,1.414218,7061.817196,19.560939,NaN,19586.107233,NaN
min,NaN,NaN,2021.000000,0.000000,3.000000,NaN,3001.000000,NaN
25%,NaN,NaN,2022.000000,5.000000,3.000000,NaN,3139.000000,NaN
50%,NaN,NaN,2023.000000,32.000000,12.000000,NaN,12139.500000,NaN
75%,NaN,NaN,2024.000000,174.000000,46.000000,NaN,46131.000000,NaN


#### Variable `anyo`

La variable `anyo` representa el periodo temporal de referencia de los datos de población. Tras analizar el rango de años disponibles en el conjunto de datos, se decide filtrar el dataset para conservar únicamente el año 2025.

Esta decisión se justifica porque las demás fuentes de datos utilizadas en el proyecto (delimitaciones territoriales y centros sanitarios) corresponden al mismo periodo temporal, lo que garantiza la coherencia temporal en la integración de las distintas fuentes y evita sesgos derivados de la comparación entre años distintos.

In [24]:
ine_clean['anyo'].unique()

array([2025, 2024, 2023, 2022, 2021])

In [25]:
# Filtrar datos para el año 2025
ine_clean['anyo'] = ine_clean['anyo'].astype(int)
ine_clean = ine_clean[ine_clean['anyo'] == 2025].reset_index(drop=True)

# Eliminar columna anyo
ine_clean = ine_clean.drop(columns=['anyo'])

ine_clean.describe(include='all')

,grupo_edad,nacionalidad,poblacion,cod_provincia,nombre_provincia,cod_municipio,nombre_municipio
count,35772,35772,35772.000000,35772.000000,35772,35772.000000,35772
unique,22,3,NaN,NaN,3,NaN,542
top,Todas las edades,Total,NaN,NaN,Valencia/València,NaN,"Atzúbia, l'"
freq,1626,11924,NaN,NaN,17556,NaN,66
mean,NaN,NaN,606.640054,26.345018,NaN,26459.627306,NaN
std,NaN,NaN,7250.895278,19.561158,NaN,19586.326248,NaN
min,NaN,NaN,0.000000,3.000000,NaN,3001.000000,NaN
25%,NaN,NaN,5.000000,3.000000,NaN,3139.000000,NaN
50%,NaN,NaN,34.000000,12.000000,NaN,12139.500000,NaN
75%,NaN,NaN,184.000000,46.000000,NaN,46131.000000,NaN


#### Variable `nacionalidad`

La variable `nacionalidad` distingue entre `Total`, `Española` y `Extranjera`.  
Al igual que ocurre con la variable `sexo`, esta desagregación no resulta esencial para el objetivo principal del análisis, centrado en la **estructura demográfica por edad y su relación con la dotación de centros sanitarios**.

Mantener esta variable:
- Multiplicaría el número de registros por municipio
- Complicaría los procesos de unión entre datasets
- Aportaría un nivel de detalle que no es necesario en esta fase del estudio

En consecuencia, se decide **filtrar los registros correspondientes a `nacionalidad = Total`** y eliminar la variable del dataset resultante.

In [26]:
ine_clean = ine_clean[ine_clean['nacionalidad'] == 'Total'].copy()
ine_clean.drop(columns='nacionalidad', inplace=True)
ine_clean.describe(include='all')

,grupo_edad,poblacion,cod_provincia,nombre_provincia,cod_municipio,nombre_municipio
count,11924,11924.000000,11924.000000,11924,11924.000000,11924
unique,22,NaN,NaN,3,NaN,542
top,Todas las edades,NaN,NaN,Valencia/València,NaN,"Atzúbia, l'"
freq,542,NaN,NaN,5852,NaN,22
mean,NaN,909.960081,26.345018,NaN,26459.627306,NaN
std,NaN,9657.909358,19.561705,NaN,19586.873819,NaN
min,NaN,0.000000,3.000000,NaN,3001.000000,NaN
25%,NaN,15.000000,3.000000,NaN,3139.000000,NaN
50%,NaN,66.000000,12.000000,NaN,12139.500000,NaN
75%,NaN,349.000000,46.000000,NaN,46131.000000,NaN


#### Variable `grupo_edad`

La variable `grupo_edad` presenta intervalos de edad quinquenales, un grupo agregado (`Todas las edades`) y un intervalo abierto para edades avanzadas (`100 y más años`). Esta estructura es coherente con la clasificación estándar del INE.

Dado que el análisis se realiza a nivel municipal, se adopta una transformación estructural de esta variable:

- Se mantiene la desagregación por grupos de edad
- Se transforma la variable `grupo_edad` de formato largo a formato ancho
- Cada grupo de edad pasa a constituir una columna independiente
- Los valores representan la población correspondiente en cada grupo de edad

Este proceso permite obtener un único registro por municipio, con la población desagregada por edades, facilitando la integración posterior con los datasets territoriales y sanitarios.

In [27]:
# Pivotar el dataset para convertir grupo_edad en columnas
ine_pivot = ine_clean.pivot_table(
    index=['cod_provincia', 'nombre_provincia', 'cod_municipio', 'nombre_municipio'],
    columns='grupo_edad',
    values='poblacion',
    aggfunc='sum',
    observed=False
).reset_index()

# Eliminar el nombre del índice de columnas
ine_pivot.columns.name = None

ine_pivot.head()


,cod_provincia,nombre_provincia,cod_municipio,nombre_municipio,100 y más años,De 0 a 4 años,De 10 a 14 años,De 15 a 19 años,De 20 a 24 años,De 25 a 29 años,...,De 55 a 59 años,De 60 a 64 años,De 65 a 69 años,De 70 a 74 años,De 75 a 79 años,De 80 a 84 años,De 85 a 89 años,De 90 a 94 años,De 95 a 99 años,Todas las edades
0,3,Alicante/Alacant,3001,"Atzúbia, l'",0.0,10.0,13.0,21.0,17.0,21.0,...,57.0,78.0,64.0,45.0,43.0,36.0,19.0,8.0,1.0,601.0
1,3,Alicante/Alacant,3002,Agost,1.0,184.0,256.0,295.0,264.0,269.0,...,419.0,401.0,299.0,232.0,211.0,137.0,95.0,68.0,16.0,5224.0
2,3,Alicante/Alacant,3003,Agres,0.0,17.0,21.0,28.0,27.0,43.0,...,62.0,56.0,34.0,43.0,33.0,26.0,9.0,7.0,3.0,625.0
3,3,Alicante/Alacant,3004,Aigües,0.0,45.0,51.0,56.0,48.0,48.0,...,91.0,96.0,80.0,56.0,50.0,38.0,11.0,2.0,3.0,1187.0
4,3,Alicante/Alacant,3005,Albatera,0.0,708.0,833.0,828.0,778.0,784.0,...,919.0,820.0,646.0,500.0,409.0,301.0,180.0,93.0,23.0,13476.0


In [28]:
# Comprobar la suma de la población por municipio
ine_pivot['total_poblacion'] = (
    ine_pivot['De 0 a 4 años'] + 
    ine_pivot['De 5 a 9 años'] + 
    ine_pivot['De 10 a 14 años'] + 
    ine_pivot['De 15 a 19 años'] + 
    ine_pivot['De 20 a 24 años'] + 
    ine_pivot['De 25 a 29 años'] + 
    ine_pivot['De 30 a 34 años'] + 
    ine_pivot['De 35 a 39 años'] + 
    ine_pivot['De 40 a 44 años'] + 
    ine_pivot['De 45 a 49 años'] + 
    ine_pivot['De 50 a 54 años'] + 
    ine_pivot['De 55 a 59 años'] + 
    ine_pivot['De 60 a 64 años'] +
    ine_pivot['De 65 a 69 años'] + 
    ine_pivot['De 70 a 74 años'] + 
    ine_pivot['De 75 a 79 años'] + 
    ine_pivot['De 80 a 84 años'] + 
    ine_pivot['De 85 a 89 años'] + 
    ine_pivot['De 90 a 94 años'] + 
    ine_pivot['De 95 a 99 años'] + 
    ine_pivot['100 y más años']
)
all(ine_pivot['total_poblacion'] == ine_pivot['Todas las edades'])

True

In [29]:
# Renombrar columna 'Todas las edades' a 'poblacion_total'
ine_pivot = ine_pivot.rename(columns={
    'Todas las edades': 'poblacion_total'
})

# Eliminar columna total_poblacion ya que es redundante
ine_pivot = ine_pivot.drop(columns=['total_poblacion'])
ine_pivot.describe(include='all')

,cod_provincia,nombre_provincia,cod_municipio,nombre_municipio,100 y más años,De 0 a 4 años,De 10 a 14 años,De 15 a 19 años,De 20 a 24 años,De 25 a 29 años,...,De 55 a 59 años,De 60 a 64 años,De 65 a 69 años,De 70 a 74 años,De 75 a 79 años,De 80 a 84 años,De 85 a 89 años,De 90 a 94 años,De 95 a 99 años,poblacion_total
count,542.000000,542,542.000000,542,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,...,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000
unique,NaN,3,NaN,542,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Valencia/València,NaN,"Atzúbia, l'",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,266,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,26.345018,NaN,26459.627306,NaN,2.413284,354.931734,516.734317,562.162362,551.269373,543.101476,...,744.994465,674.153137,582.151292,476.407749,408.208487,295.754613,167.658672,86.959410,21.619926,10009.560886
std,19.578954,NaN,19604.145835,NaN,12.695890,1498.760716,2058.244925,2300.419467,2483.479662,2674.789191,...,3130.386362,2841.301131,2463.809427,2075.738705,1833.932374,1336.866457,778.643578,410.751696,107.716508,43098.380733
min,3.000000,NaN,3001.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21.000000
25%,3.000000,NaN,3139.250000,NaN,0.000000,12.000000,17.000000,18.250000,21.000000,21.000000,...,40.000000,40.000000,36.000000,28.000000,24.250000,21.000000,13.000000,6.000000,1.000000,504.000000
50%,12.000000,NaN,12139.500000,NaN,0.000000,50.000000,69.000000,75.000000,72.000000,73.500000,...,126.500000,123.000000,100.000000,82.500000,71.500000,60.000000,34.000000,19.000000,4.000000,1571.500000
75%,46.000000,NaN,46130.750000,NaN,2.000000,264.000000,397.500000,417.500000,371.000000,383.500000,...,571.750000,483.750000,421.250000,341.750000,272.750000,205.500000,116.500000,65.000000,14.000000,7698.500000


### Exportación del dataset de población

Una vez finalizado el proceso de limpieza, transformación y filtrado del conjunto de datos de población del INE, se procede a guardar el dataset resultante en la carpeta `data/processed`. Este conjunto de datos limpio y estructurado será utilizado posteriormente en la fase de integración con las demás fuentes de información del proyecto.

El archivo se almacena en formato CSV para facilitar su reutilización en futuros análisis o proyectos.

In [30]:
ine_pivot.sample(5)

,cod_provincia,nombre_provincia,cod_municipio,nombre_municipio,100 y más años,De 0 a 4 años,De 10 a 14 años,De 15 a 19 años,De 20 a 24 años,De 25 a 29 años,...,De 55 a 59 años,De 60 a 64 años,De 65 a 69 años,De 70 a 74 años,De 75 a 79 años,De 80 a 84 años,De 85 a 89 años,De 90 a 94 años,De 95 a 99 años,poblacion_total
75,3,Alicante/Alacant,3076,Guardamar del Segura,1.0,618.0,903.0,940.0,855.0,839.0,...,1400.0,1392.0,1273.0,1101.0,988.0,623.0,341.0,146.0,38.0,18541.0
512,46,Valencia/València,46237,Tavernes Blanques,4.0,312.0,497.0,579.0,509.0,528.0,...,722.0,653.0,631.0,484.0,381.0,257.0,136.0,87.0,19.0,9789.0
286,46,Valencia/València,46011,Alberic,0.0,474.0,604.0,677.0,613.0,603.0,...,837.0,736.0,627.0,478.0,372.0,300.0,210.0,76.0,24.0,11109.0
262,12,Castellón/Castelló,12131,Villamalur,0.0,2.0,0.0,2.0,5.0,8.0,...,5.0,7.0,12.0,10.0,8.0,5.0,1.0,1.0,0.0,99.0
47,3,Alicante/Alacant,3048,Callosa d'en Sarrià,3.0,309.0,421.0,444.0,492.0,455.0,...,606.0,545.0,468.0,353.0,288.0,230.0,108.0,83.0,26.0,8301.0


In [31]:
ine_pivot.to_csv('../data/processed/INE-poblacion-grupo-edad-clean.csv', index=False)